In [ ]:
# 📍 STEP 1: Install required packages
!pip install flask flask-cors pyngrok sentence-transformers faiss-cpu beautifulsoup4 PyPDF2 serpapi google-generativeai --quiet

# 📍 STEP 2: Imports
import os
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from sentence_transformers import SentenceTransformer
import faiss
import PyPDF2
import requests
from bs4 import BeautifulSoup
from serpapi import GoogleSearch
import google.generativeai as genai

# 📍 STEP 3: Set Gemini + SerpAPI Keys
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY"
SERPAPI_API_KEY = "YOUR_SERPAPI_KEY"
genai.configure(api_key=GEMINI_API_KEY)

# 📍 STEP 4: Load embedding model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# 📍 STEP 5: Utility functions
def extract_text_from_url(url):
    try:
        r = requests.get(url, timeout=5)
        soup = BeautifulSoup(r.text, 'html.parser')
        return " ".join(p.text for p in soup.find_all("p"))
    except:
        return ""

def extract_text_from_pdf(pdf_path):
    try:
        reader = PyPDF2.PdfReader(open(pdf_path, "rb"))
        return "\n".join(p.extract_text() for p in reader.pages[:3])
    except:
        return ""

def google_search_snippets(query):
    params = {
        "q": query,
        "api_key": SERPAPI_API_KEY,
        "num": 3
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    return "\n".join([r.get("snippet", "") for r in results.get("organic_results", [])])

def build_rag_context(query, sources):
    embeds = embed_model.encode(sources, show_progress_bar=False)
    index = faiss.IndexFlatL2(embeds.shape[1])
    index.add(embeds)

    q_embed = embed_model.encode([query])
    D, I = index.search(q_embed, 3)
    return "\n".join([sources[i] for i in I[0]])

def generate_notes_gemini(prompt):
    model = genai.GenerativeModel("gemini-pro")
    response = model.generate_content(prompt)
    return response.text

# 📍 STEP 6: Flask server
app = Flask(__name__)
CORS(app)

@app.route("/generate", methods=["POST"])
def generate():
    data = request.json
    question = data["question"]
    options = data["options"]
    urls = data["urls"]
    book_texts = data["book_texts"]
    use_google = data["use_google"]

    # Context Collection
    sources = []

    for url in urls:
        sources.append(extract_text_from_url(url))

    for book_content in book_texts:
        sources.append(book_content)

    if use_google:
        sources.append(google_search_snippets(question))

    # RAG
    context = build_rag_context(question, sources)

    prompt = f"""
You are an expert tutor.

Question: {question}
Options:
A. {options[0]}
B. {options[1]}
C. {options[2]}
D. {options[3]}

Context:
{context}

Instructions:
1. Explain the concept behind the question.
2. Analyze each option.
3. If the question involves a chemical test, include its discovery, reagents, and interaction.
4. If it's about a mechanism, describe every step clearly.
"""

    result = generate_notes_gemini(prompt)
    return jsonify({"notes": result})

# 📍 STEP 7: Launch server using ngrok
public_url = ngrok.connect(5000)
print("API running at:", public_url)

app.run(port=5000)
